In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import subprocess
import scipy.io as sio
import cv2

# LBP  
https://en.wikipedia.org/wiki/Local_binary_patterns

In [8]:
def LBP(img, patch_size=3):
#     img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    
    H, W = img.shape
#     print img.shape
    s0, s1 = img.strides
    nH = H - patch_size + 1
    nW = W - patch_size + 1
    
    shp = patch_size, patch_size, nH, nW
    strd = s0, s1, s0, s1
    out_view = np.lib.stride_tricks.as_strided(img, shape=shp, strides=strd)
#     print out_view.shape
    img_patch = out_view.reshape(patch_size*patch_size, nH, nW).transpose(1,2,0)
#     print img_patch.shape
    base = np.array([128, 64, 32, 1, 0, 16, 2, 4, 8])
#     print base.shape
    mask = [img_patch > img_patch[:, :, 4].reshape(nH, nW, 1)]
    mask = np.array(mask)[0]
#     print mask.shape
    LBP_img = np.sum(mask*base, axis = 2)
#     print LBP_img.shape
    return LBP_img

# For NO.1 ~ NO.10 people , each one has 22 face images in this dataset with LBP.

In [9]:
def extract_data_LBP():
    
    if os.path.exists('./data2') == False:
        print "'./data2' path doesn't exist"
        print "Creat path './data2'"
        os.mkdir('./data2')
    else:
        X = np.load('./data2/X_train.npy')
        y = np.load('./data2/y_label.npy')
        print 'Data is existing, success load data:'
        print 'Training data: '+str(X.shape)
        print 'Label: '+str(y.shape)
        return X, y
    
    print 'Extracting data...'
    PATH = './CroppedYale10'
    print 'Check all image files\' package' 
    print subprocess.check_output(["ls", PATH]).decode('utf8')
    all_face_package_path = os.listdir(PATH)
    FLAG = True
    for i, face_path in enumerate(all_face_package_path):
        Face_one_person = os.path.join(PATH, face_path)
    #     print subprocess.check_output(["ls", Face_one_person]).decode('utf8')
#         all_face_one_person = os.listdir(Face_one_person)
        with open(Face_one_person+'/'+face_path+'_P00.info') as f: 
            all_face_one_person = f.read().splitlines()
        for j, one_face in enumerate(all_face_one_person):
            if j==0:
                continue
            face_img_path = os.path.join(Face_one_person, one_face)
            img = cv2.imread(face_img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            face = cv2.resize(img, (168,192))
#             face = saliency_fft(face)
#             face = LBP(img, 3)
            face = LBP(face, 3)
            if j < 10:
                print j
                plt.imshow(face, cmap='gray')
                plt.axis('off')
                plt.show()
            H, W = face.shape
            print face_img_path
            print face.shape
            face = face.reshape(-1)
            if FLAG == True:
                X = np.zeros((1, H*W))
                y = np.zeros(1, np.int)
                X = X + face
                FLAG = False
                continue
            print i,j
            X = np.vstack((X, face))
            y = np.vstack((y,np.array(i)))
    np.save('./data2/X_train.npy', X)
    np.save('./data2/y_label.npy', y)
    return X, y

# All face images in this dataset, resize to $\frac{1}{4}$ resolution compare to original image.

In [10]:
def extract_data():
    
    if os.path.exists('./data') == False:
        print "./data path doesn't exist"
        print "Creat path ./data"
        os.mkdir('./data')
    else:
        X = np.load('./data/X_train.npy')
        y = np.load('./data/y_label.npy')
        print 'Data is existing, success load data:'
        print 'Training data: '+str(X.shape)
        print 'Label: '+str(y.shape)
        return X, y
    
    print 'Extracting data...'
    PATH = './CroppedYale'
    print 'Check all image files\' package' 
    print subprocess.check_output(["ls", PATH]).decode('utf8')
    all_face_package_path = os.listdir(PATH)
    FLAG = True
    for i, face_path in enumerate(all_face_package_path):
        Face_one_person = os.path.join(PATH, face_path)
    #     print subprocess.check_output(["ls", Face_one_person]).decode('utf8')
        all_face_one_person = os.listdir(Face_one_person)
        for j, one_face in enumerate(all_face_one_person):
            face_img_path = os.path.join(Face_one_person, one_face)
            img = cv2.imread(face_img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue
            face = cv2.resize(img, (42,48))
#             face = saliency_fft(face)
#             face = LBP(img, 3)
#             face = LBP(face, 3)
            if j < 10:
                print j
                plt.imshow(face, cmap='gray')
                plt.axis('off')
                plt.show()
            H, W = face.shape
            print face_img_path
            print face.shape
            face = face.reshape(-1)
            if FLAG == True:
                X = np.zeros((1, H*W))
                y = np.zeros(1, np.int)
                X = X + face
                FLAG = False
                continue
            print i,j
            X = np.vstack((X, face))
            y = np.vstack((y,np.array(i)))
    np.save('./data/X_train.npy', X)
    np.save('./data/y_label.npy', y)
    return X, y

In [15]:
X, y = extract_data_LBP()

Data is existing, success load data:
Training data: (220, 31540)
Label: (220, 1)


In [16]:
samePicDis = np.sqrt(np.sum((X[0:22] - X[1])**2, axis=1))
print samePicDis
samePicDis = np.sqrt(np.sum((X[22:44] - X[1])**2, axis=1))
print samePicDis
# for i in range(10):
#     samePicDis = np.sqrt(np.sum((X[i*23:i*23+23] - X[i*23])**2, axis=1))
#     print np.sum(samePicDis<18000)

[13058.374516       0.         10884.78084299 14726.71769268
 16723.21640714 18047.94993898 18209.29471451 18787.04569644
 17979.65322246 17655.92266068 17105.39593228 16744.81149491
 15891.28934354 18314.41699864 20174.10374713 17239.94286533
 17141.38553326 18129.42232946 19011.1079635  18897.97481213
 17937.27738538 17642.08343705]
[22052.16449694 21992.09748978 26016.35116614 25957.77403785
 22115.51177341 22346.56685041 22463.10495012 22494.3253733
 22238.20386182 22051.18196379 21989.01246077 21935.04752673
 21869.56403315 22203.03875599 22361.73542013 22069.54091049
 21929.48387446 21976.59985075 22189.70276953 22093.44522704
 21865.91473047 21897.5760074 ]


In [17]:
# normalization
X = X / 255
# X = X - X.mean(axis=1).reshape(-1,1)

In [18]:
# number of classes
K = 10

N, D = X.shape
# shuffle data
np.random.seed(520)
X_train = np.random.permutation(X)
np.random.seed(520)
y_label = np.random.permutation(y)

In [23]:
X_train_expend = X_train.repeat(K, axis = 0)
# init K*D center
K_centers = np.random.rand(K,D)
# K_centers = X_train[:10]
print X_train_expend.shape, K_centers.shape

(2200, 31540) (10, 31540)


In [24]:
y_train = np.zeros((N,1), np.int)
iters = 15
for i in range(iters):
#     X_train_expend = X_train.repeat(K, axis = 0)
#     old_K_centers = np.copy(K_centers)
    K_centers += 0.2*np.random.rand(K,D)
    K_centers_expend = np.tile(K_centers, (N,1))
    dis = np.sum((X_train_expend - K_centers_expend)**2, axis = 1)
    dis = dis.reshape(N, K)
    
    min_idx = np.argmin(dis, axis = 1)
#     old_idx = np.copy(min_idx)
#     print min_idx
#     print y_label.reshape(-1)

    for j in range(K):
#         print 'j: '+str(j)
        j_idx = np.argwhere(min_idx==j)
#         print j_idx.reshape(-1)
#         print j_idx.shape, j_idx
        if j_idx.shape[0] == 0:
            K_centers[j] = np.random.rand(1,D)
            continue
#         print y_label[j_idx].reshape(-1)
        count = np.bincount(y_label[j_idx].reshape(-1))
#         print np.argmax(count)
        y_train[j_idx] = np.argmax(count)
#         print y_train[j_idx].reshape(-1)
        all_j = X_train[j_idx.reshape(-1),:]
    #     print all_j.shape
        K_centers[j] = np.mean(all_j, axis = 0).reshape(1,-1)
#     print y_train.reshape(-1)
#     print y_label.reshape(-1)
    acc = np.mean(y_train.reshape(-1) == y_label.reshape(-1))
    print 'iter: '+str(i)+' acc: '+ str(acc)

iter: 0 acc: 0.31363636363636366
iter: 1 acc: 0.5636363636363636
iter: 2 acc: 0.6681818181818182
iter: 3 acc: 0.7318181818181818
iter: 4 acc: 0.75
iter: 5 acc: 0.75
iter: 6 acc: 0.75
iter: 7 acc: 0.75
iter: 8 acc: 0.75
iter: 9 acc: 0.75
iter: 10 acc: 0.75
iter: 11 acc: 0.75
iter: 12 acc: 0.75
iter: 13 acc: 0.75
iter: 14 acc: 0.75


# All image K-means, 38 clusters

In [25]:
X, y = extract_data()

Data is existing, success load data:
Training data: (2470, 7708)
Label: (2470, 1)


In [26]:
samePicDis = np.sqrt(np.sum((X[0:65] - X[1])**2, axis=1))
print samePicDis
samePicDis = np.sqrt(np.sum((X[65:130] - X[1])**2, axis=1))
print samePicDis
# for i in range(10):
#     samePicDis = np.sqrt(np.sum((X[i*23:i*23+23] - X[i*23])**2, axis=1))
#     print np.sum(samePicDis<18000)

[11005.50912044     0.         10441.18039304 10571.12212587
 11132.58837827 10779.48166657 10392.09598685 10101.10746404
 10684.71038447 10429.01975259 11106.69266704  9496.80756886
 11384.3170634  10884.2879418   8807.16492408 10644.29607818
 10999.05836879 10855.13597335 10851.38235434 10940.63869251
 10635.83066808 11753.74753004  8479.7849619  10665.03895914
 10828.00858884 11007.11501712  8997.70026173 10853.91652815
 10971.75209344 10806.58100418  8569.9843057  10301.17769966
 10307.24352094 10721.08189503 10952.18695056 11419.70800853
 11183.25909563 11036.84483899 10816.72339482 11140.97441878
 11617.84390496  9872.07435142  9592.55132903 10916.66496692
  9439.68707108 11204.46942073 10832.17295837 10064.59154661
 11025.7419705  10983.88433115  8372.71574819 11064.5717947
 10738.09708468 11113.12489807  9670.87426244 10495.56911273
 11048.08309165 11057.63672762 11079.42480456 10526.56900419
 10867.67371612 10816.58684614  8086.5340536   9193.60054603
 10477.70113145]
[11472.6

In [27]:
# normalization
X /= 255
# number of classes
K = 38

N, D = X.shape
# shuffle data
np.random.seed(520)
X_train = np.random.permutation(X)
np.random.seed(520)
y_label = np.random.permutation(y)

In [30]:
X_train_expend = X_train.repeat(K, axis = 0)
# init K*D center
K_centers = np.random.rand(K,D)
# K_centers = X_train[:10]
print X_train_expend.shape, K_centers.shape

(93860, 7708) (38, 7708)


In [31]:
y_train = np.zeros((N,1), np.int)
iters = 30
for i in range(iters):
#     X_train_expend = X_train.repeat(K, axis = 0)
#     old_K_centers = np.copy(K_centers)
    K_centers += 0.2*np.random.rand(K,D)
    K_centers_expend = np.tile(K_centers, (N,1))
    dis = np.sum((X_train_expend - K_centers_expend)**2, axis = 1)
    dis = dis.reshape(N, K)
    
    min_idx = np.argmin(dis, axis = 1)
#     old_idx = np.copy(min_idx)
#     print min_idx
#     print y_label.reshape(-1)

    for j in range(K):
#         print 'j: '+str(j)
        j_idx = np.argwhere(min_idx==j)
#         print j_idx.reshape(-1)
#         print j_idx.shape, j_idx
        if j_idx.shape[0] == 0:
            K_centers[j] = np.random.rand(1,D)
            continue
#         print y_label[j_idx].reshape(-1)
        count = np.bincount(y_label[j_idx].reshape(-1))
#         print np.argmax(count)
        y_train[j_idx] = np.argmax(count)
#         print y_train[j_idx].reshape(-1)
        all_j = X_train[j_idx.reshape(-1),:]
    #     print all_j.shape
        K_centers[j] = np.mean(all_j, axis = 0).reshape(1,-1)
#     print y_train.reshape(-1)
#     print y_label.reshape(-1)
    acc = np.mean(y_train.reshape(-1) == y_label.reshape(-1))
    print 'iter: '+str(i)+' acc: '+ str(acc)

iter: 0 acc: 0.11700404858299596
iter: 1 acc: 0.20080971659919028
iter: 2 acc: 0.25020242914979757
iter: 3 acc: 0.28016194331983807
iter: 4 acc: 0.29392712550607286
iter: 5 acc: 0.29676113360323886
iter: 6 acc: 0.2975708502024291
iter: 7 acc: 0.3
iter: 8 acc: 0.2987854251012146
iter: 9 acc: 0.3
iter: 10 acc: 0.302834008097166
iter: 11 acc: 0.30364372469635625
iter: 12 acc: 0.30526315789473685
iter: 13 acc: 0.308502024291498
iter: 14 acc: 0.31093117408906884
iter: 15 acc: 0.31093117408906884
iter: 16 acc: 0.31497975708502024
iter: 17 acc: 0.31497975708502024
iter: 18 acc: 0.3145748987854251
iter: 19 acc: 0.31497975708502024
iter: 20 acc: 0.3153846153846154
iter: 21 acc: 0.3157894736842105
iter: 22 acc: 0.3157894736842105
iter: 23 acc: 0.3153846153846154
iter: 24 acc: 0.31497975708502024
iter: 25 acc: 0.3145748987854251
iter: 26 acc: 0.31497975708502024
iter: 27 acc: 0.31619433198380564
iter: 28 acc: 0.31619433198380564
iter: 29 acc: 0.31619433198380564
